<a href="https://colab.research.google.com/github/tara-shukla/jrw/blob/main/multi_gpt_model_jrw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install numba

#from numba import cuda
#device = cuda.get_current_device()
#device.reset()


#PYTORCH_CUDA_ALLOC_CONF=expandable_segments = True


In [ ]:
!pip install datasets
!pip install transformers[torch]
! pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 43.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-no

In [ ]:
from datasets import load_dataset
import json
import os
import glob



#get train data file
!wget -q -O train.txt "https://docs.google.com/uc?export=download&id=1cQBB074e3-xffKNuDgUok5M_u7iuykYZ"

#get train corrected data file
!wget -q -O test.txt 'https://docs.google.com/uc?export=download&id=1P0IJRSSwxgAMPJP98bHnJNgzHqG-PfRl'

#get val data file
!wget -q -O val.txt 'https://docs.google.com/uc?export=download&id=18AvCV0nwvxSUA8U102enCqzdsLHsf04E'

#splitting text data and putting into json format
def formatdata(filename):
    with open((filename+".txt"), "r", encoding = 'utf-8') as file:
        data = []
        for line in file:
            error_sentence, correct_sentence = line.strip().split('\t')
            data.append({'error_sentence': error_sentence, 'correct_sentence': correct_sentence})

        indexed_data = []
        for index, pair in enumerate(data):
            indexed_pair = {'index': index, **pair}
            indexed_data.append(indexed_pair)

    json_data = json.dumps(indexed_data, ensure_ascii=False, indent=4)

    # Write JSON data
    with open((filename+".json"), "w", encoding = 'utf-8') as json_file:
        json_file.write(json_data)


formatdata("train")
formatdata("test")
formatdata("val")

#loading dataset
raw_datasets = load_dataset("json", data_files={'train': 'train.json', 'test': 'test.json', 'val':'val.json'})


raw_datasets = raw_datasets.filter(lambda x: x["correct_sentence"] is not None)
raw_datasets = raw_datasets.filter(lambda x: x["error_sentence"] is not None)
data_sample = raw_datasets["train"].shuffle(seed=63).select(range(1000))
# Peek at data
data_sample[:3]

raw_train_dataset = raw_datasets["train"]
#need to index like in tutorial? labels??
raw_train_dataset[0]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/19898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4265 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4264 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4265 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4264 [00:00<?, ? examples/s]

{'index': 0,
 'correct_sentence': '사람들이 정년 연장은 고령화 사회의 해결 방법이라고 생각했다.',
 'error_sentence': '사람들이 정년을 연장은 고령화 사회의 해결 방법이라고 생각했다.'}

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True
).to(device='cuda', non_blocking=True)

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/839 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/12.3G [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 19.06 MiB is free. Process 7129 has 14.73 GiB memory in use. Of the allocated memory 14.62 GiB is allocated by PyTorch, and 9.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def preprocess_function(examples):
    inputs = examples["error_sentence"]
    targets = examples["correct_sentence"]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=128, truncation=True
    )
    return model_inputs


tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

tokenized_datasets

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()
batch["decoder_input_ids"]

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"outputdir_me",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    push_to_hub=False,
    report_to="wandb"
)

In [ ]:
from nltk.translate.gleu_score import sentence_gleu as gleu_score
from sklearn.metrics import precision_recall_fscore_support
!pip install evaluate
import evaluate
!pip install sacrebleu

gleu_metric = evaluate.load("google_bleu")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

bleu_metric = evaluate.load("sacrebleu")
metrics = [gleu_metric, precision_metric, recall_metric, f1_metric]


def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    gleu = gleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bleu = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)


    #for p, l in zip(decoded_preds, decoded_labels):
        #print(f"pred: {p}, label: {l}")

    precision, recall, f1, _ = precision_recall_fscore_support(decoded_labels, decoded_preds, average='weighted')
    #gleu = gleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #precision = precision_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #recall = recall_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #f1=f1_metric.compute(predictions=decoded_preds, references=decoded_labels)



    return {
      "bleu": bleu["score"],
      "gleu": gleu["google_bleu"],
      "f1": f1,
      "precision": precision,
      "recall": recall
      }


In [ ]:
!pip install wandb

In [ ]:
#make trainer
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

trainer.evaluate(max_length=128)

In [ ]:
trainer.train()


In [ ]:
trainer.evaluate(max_length=128)

In [ ]:
#next(model.parameters()).is_cuda

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

In [ ]:
import torch

def finetuned_generate(text, model, tokenizer):
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
  corrected_ids = model.generate(torch.tensor([input_ids]),
                                  max_length=128,
                                  eos_token_id=1, num_beams=4,
                                  early_stopping=True, repetition_penalty=2.0)

  #output_text = tokenizer.decode(corrected_ids.squeeze().tolist(), skip_special_tokens=True)
  #return output_text
  output_texts = []
  for ids in corrected_ids:
      decoded_text = tokenizer.decode(ids, skip_special_tokens=True)
      sentences = decoded_text.split('.')  # Split into sentences
      for sentence in sentences:
          if sentence.strip():  # Check if the sentence is not empty
              output_texts.append(sentence.strip()+".")



  return output_texts


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to('cpu')
text = '한국어이어렵다고 생각하고 잇다.'
output_text=finetuned_generate(text, model, tokenizer)
print(output_text)


In [ ]:


# Save the model to Google Drive
model.save_pretrained('/content/drive/My Drive/mbart_model')
tokenizer.save_pretrained('/content/drive/My Drive/mbart_model')

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_path = "/content/drive/My Drive/mbart_model"
tokenizer_path = "/content/drive/My Drive/mbart_model"
config_path = "/content/drive/My Drive/mbart_model"


model = MBartForConditionalGeneration.from_pretrained(model_path)
tokenizer = MBart50TokenizerFast.from_pretrained(tokenizer_path, src_lang="ko_KR", tgt_lang="ko_KR")



In [ ]:
import torch
from nltk.translate.gleu_score import sentence_gleu as gleu_score
import nltk
nltk.download('punkt')

def calculate_gleu_score(candidate_text, reference_text):

  return gleu_score([reference_text], candidate_text)

def finetuned_generate(text, model, tokenizer, reference_text):
    raw_input_ids = tokenizer.encode(text)
    input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
    corrected_ids = model.generate(torch.tensor([input_ids]),
                                    max_length=128,
                                    eos_token_id=1, num_beams=4,
                                    early_stopping=True, repetition_penalty=2.0,
                                    num_return_sequences=4)  # Change the number of return sequences as needed


    output_texts = []
    for ids in corrected_ids:
        decoded_text = tokenizer.decode(ids, skip_special_tokens=True)
        sentences = decoded_text.split('.')  # Split into sentences
        for sentence in sentences:
            if sentence.strip():  # Check if the sentence is not empty
                output_texts.append(sentence.strip()+".")




    best_gleu_score = float('-inf')
    best_candidate = None
    for sentence in output_texts:

        gleu_score = calculate_gleu_score(sentence, reference_text)
        if gleu_score > best_gleu_score:
            best_gleu_score = gleu_score
            best_candidate = sentence

    return best_candidate

In [ ]:
text = '한국어이어렵다고 생각하고 잇다.'
corrected = "한국어가 어렵다고 생각하고 있다."
output_text=finetuned_generate(text, model, tokenizer, corrected)
print(output_text)

In [ ]:
output_file_path = "mbart_corrected_dataset.txt"

with open(output_file_path, "w", encoding="utf-8") as output_file:
    # Iterate through each tuple in the test dataset
    for sentences in raw_datasets["test"]:
        correct_sentence = sentences["correct_sentence"]
        error_sentence = sentences["error_sentence"]
        index = sentences["index"]
        corrected_sentence = finetuned_generate(error_sentence, model, tokenizer, correct_sentence)

        # Write Corrected Sentence and Index to the Text File
        #output_file.write(f"Index: {index}\n")
        #output_file.write(f"Correct Sentence: {correct_sentence}\n")
        output_file.write(corrected_sentence)
        print(corrected_sentence)
        output_file.write("\n")

# Inform when all sentences are corrected and saved
print("All sentences corrected and saved to:", output_file_path)